In [16]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import time
import torch
import torchvision
import csgm
import dcgan
import skimage
import anchor_image
import image_utils
import importlib
import time
import pickle

In [13]:
importlib.reload(anchor_image)

<module 'anchor_image' from '/home/kurtis/Desktop/anchor/ImageAnchors/anchor_image.py'>

In [4]:
samples = {0: [3],
        1: [2],
        2: [1],
        3: [18],
        4: [4],
        5: [8],
        6: [11],
        7: [0],
        8: [61],
        9: [7]}

In [5]:
mnist_test = torchvision.datasets.MNIST('./', train=False, transform=lambda x: np.array(x)/255.,
                                   target_transform=None, download=True)

## Setting up

In [6]:
parameters = {
    'dummy': {'dataset': mnist_test, 'dummys': mnist_test},
    'batch': {'G': lambda x: dcgan.load_generator(eval=True), 'dataset': mnist_test, 'batch_norm': False},
    'batchBN': {'G': lambda x: dcgan.load_generator(eval=False),'dataset': mnist_test, 'batch_norm': True}
}

In [7]:
net = dcgan.Net()
net.load_state_dict(torch.load('mnist_cnn.pt'))

In [8]:
def predict_fn(images):
    try:
        images = torch.from_numpy(images).type(torch.FloatTensor)
    except:
        pass
    return net(images.view(images.shape[0],1,28,28)).data.numpy()

## Collect data Function

In [42]:
times = {
    key: {i: [] for i in range(10)
         }  for key in parameters}

precisions = {key:{i: [] for i in range(10)}
                 for key in parameters
             }

In [43]:
def collectData(parameter, times, precisions, n_trials = 1):
    currTime = times[parameter]
    currPrecision = precisions[parameter]
    for c in samples:
        sample = samples[c][0]
        for i in range(n_trials):
            image = mnist_test[sample][0]
            explainer = anchor_image.AnchorImageMNIST([],'yeet', **parameters[parameter])
            start = time.time()
            segments, exp = explainer.explain_instance(image,predict_fn,verbose=True)
            end = time.time()
            currTime[c].append(end-start)
            currPrecision[c].append(exp)
            with open(parameter+'_times.pck','wb') as f:
                pickle.dump(currTime,f)
            with open(parameter+'_precisions.pck','wb') as f:
                pickle.dump(currPrecision,f)
    

## Collect data

In [44]:
parameter = 'dummy'

In [45]:
collectData(parameter,times,precisions,n_trials = 3)

found max_dist of  3  created  15  segments
True pred 0
Taking coverage data...
Checkpoint 1
Making tuples...
Best: 17 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 0 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 26 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 34 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 2 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 12 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 4 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 30 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 10 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 27 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 13 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 11 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 29 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 25 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 20 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 31 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 26 (mean:0.5445544554, n: 101, lb:0.2696) Worst: 32 (mean:0.0000, n: 1, ub:1.0000) B = 0.73
Best: 26 (mean:0.4825870647

Best: 26 (mean:0.4818976280, n: 801, lb:0.3795) Worst: 9 (mean:0.0000, n: 1, ub:1.0000) B = 0.62
Best: 26 (mean:0.4805771365, n: 901, lb:0.3837) Worst: 27 (mean:0.0000, n: 1, ub:1.0000) B = 0.62
Best: 26 (mean:0.4895104895, n: 1001, lb:0.3970) Worst: 8 (mean:0.0000, n: 1, ub:1.0000) B = 0.60
Best: 26 (mean:0.4895549500, n: 1101, lb:0.4011) Worst: 25 (mean:0.0000, n: 1, ub:1.0000) B = 0.60
Best: 26 (mean:0.4870940883, n: 1201, lb:0.4022) Worst: 1 (mean:0.0000, n: 1, ub:1.0000) B = 0.60
Best: 26 (mean:0.4911606457, n: 1301, lb:0.4093) Worst: 31 (mean:0.0000, n: 1, ub:1.0000) B = 0.59
Best: 26 (mean:0.4910778016, n: 1401, lb:0.4120) Worst: 24 (mean:0.0000, n: 1, ub:1.0000) B = 0.59
Best: 26 (mean:0.5016655563, n: 1501, lb:0.4249) Worst: 20 (mean:0.0000, n: 1, ub:1.0000) B = 0.58
Best: 26 (mean:0.5046845721, n: 1601, lb:0.4302) Worst: 19 (mean:0.0000, n: 1, ub:1.0000) B = 0.57
Best: 26 (mean:0.5102880658, n: 1701, lb:0.4378) Worst: 18 (mean:0.0000, n: 1, ub:1.0000) B = 0.56
Best: 26 (mean:

Best: 16 (mean:0.1683168317, n: 101, lb:0.0326) Worst: 1 (mean:0.0000, n: 1, ub:1.0000) B = 0.97
Best: 1 (mean:0.1683168317, n: 101, lb:0.0320) Worst: 14 (mean:0.0000, n: 1, ub:1.0000) B = 0.97
Best: 1 (mean:0.1890547264, n: 201, lb:0.0693) Worst: 13 (mean:0.0000, n: 1, ub:1.0000) B = 0.93
Best: 1 (mean:0.1860465116, n: 301, lb:0.0833) Worst: 12 (mean:0.0000, n: 1, ub:1.0000) B = 0.92
Best: 1 (mean:0.1745635910, n: 401, lb:0.0855) Worst: 2 (mean:0.0000, n: 1, ub:1.0000) B = 0.91
Best: 1 (mean:0.1596806387, n: 501, lb:0.0819) Worst: 11 (mean:0.0000, n: 1, ub:1.0000) B = 0.92
Best: 1 (mean:0.1597337770, n: 601, lb:0.0872) Worst: 10 (mean:0.0000, n: 1, ub:1.0000) B = 0.91
Best: 10 (mean:0.1881188119, n: 101, lb:0.0382) Worst: 9 (mean:0.0000, n: 1, ub:1.0000) B = 0.96
Best: 16 (mean:0.1592039801, n: 201, lb:0.0498) Worst: 3 (mean:0.0000, n: 1, ub:1.0000) B = 0.95
Best: 3 (mean:0.1980198020, n: 101, lb:0.0420) Worst: 15 (mean:0.0000, n: 1, ub:1.0000) B = 0.96
Best: 3 (mean:0.2139303483, n: 

KeyboardInterrupt: 

In [46]:
with open('dummy_precisions.pck', 'rb') as f:
    res = pickle.load(f)

In [47]:
res

{0: [[(26, '', 0.5273679200380771, [], 0),
   (18, '', 0.8637532133676092, [], 0),
   (8, '', 1.0, [], 0)],
  [(26, '', 0.53218712514994, [], 0),
   (18, '', 0.8824742268041237, [], 0),
   (8, '', 1.0, [], 0)],
  [(26, '', 0.5116623472787857, [], 0),
   (18, '', 0.8738049713193117, [], 0),
   (8, '', 1.0, [], 0)]],
 1: [[(3, '', 0.2436738519212746, [], 0),
   (8, '', 0.312, [], 0),
   (12, '', 0.45316253002401924, [], 0)],
  [(3, '', 0.2303498409813721, [], 0),
   (8, '', 0.31791421362489486, [], 0),
   (12, '', 0.43587946741415556, [], 0)]],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: []}

In [39]:
res

{0: [[(26, '', 0.5445458496305954, [], 0),
   (18, '', 0.8918918918918919, [], 0),
   (8, '', 1.0, [], 0)],
  [(26, '', 0.4956268221574344, [], 0),
   (18, '', 0.8444924406047516, [], 0),
   (8, '', 1.0, [], 0)]],
 1: [[(3, '', 0.23950419832067174, [], 0),
   (8, '', 0.3297161936560935, [], 0),
   (12, '', 0.44216089254257196, [], 0)],
  [(3, '', 0.2438118811881188, [], 0),
   (5, '', 0.3261802575107296, [], 0),
   (16, '', 0.43095667870036103, [], 0)]],
 2: [[(13, '', 0.5682878081279147, [], 0),
   (8, '', 0.7701863354037267, [], 0),
   (10, '', 0.9935897435897436, [], 0)],
  [(13, '', 0.5602931379080613, [], 0),
   (8, '', 0.7883495145631068, [], 0),
   (10, '', 1.0, [], 0)]],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: []}